# Station characterisation

In this notebook, bicycle counting stations are characterised based on their temporal traffic patterns. The aim is to distinguish stations dominated by work-related (utilitarian) traffic from those primarily used for recreational travel, while explicitly allowing for intermediate cases.

We define three usage types:

- Recreational: relatively high weekend traffic, weak weekend drop, strong seasonal variation

- Utilitarian: pronounced weekday double peaks, strong weekend drop, weak seasonal variation

- Mixed: a combination of recreational and utilitarian characteristics

Important: These patterns are not strict rules but guiding assumptions. For example, a recreational station may still exhibit weekday double peaks due to commuter traffic. Such behaviour is not contradictory if other dominant properties justify its classification as recreational.

In [ ]:
from data_io.loader.data_loader import DataLoader

dl = DataLoader(city="Stadt_Heidelberg")


import polars as pl

pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)

## Indices
To compare station behaviour across different temporal scales, we use normalized indices on hourly, daily, and monthly levels. Normalisation ensures that stations with different absolute traffic volumes remain comparable.

To illustrate the output of each index, we use an arbitrary bicycle counting station in Heidelberg

In [ ]:
station = dl.get_bicyle_stations()[0]

**Daily Mean Count**

The daily mean count $\bar{C}_{24h}$ is defined as the total daily traffic divided by the number of observed days.

In [ ]:
from analysis.visualization.characterisation.indices import daily_mean_count

daily_mean_count(loader=dl, station_name=station)

**Hourly index**

The hourly index measures the relative traffic intensity of each hour compared to the daily mean:

$$
\begin{align*}
I_{h} = \frac{\bar{C}_{1h}}{\bar{C}_{24h}}
\end{align*}
$$

This highlights typical diurnal patterns, such as low night-time counts and increased traffic during morning and evening hours.

In [ ]:
from analysis.visualization.characterisation.indices import hourly_index

hourly_index(loader=dl, station_name=station, channel="channels_all")

**Daily Index**

The daily index captures day-to-day variability, normalised by the overall daily mean:

$$
\begin{align*}
I_{d} = \frac{\bar{C}_{1d}}{\bar{C}_{24h}}
\end{align*}
$$

In [ ]:
from analysis.visualization.characterisation.indices import daily_index

daily_index(loader=dl, station_name=station, channel="channels_all")

**Monthly Index**

The monthly index reflects seasonal variation by comparing average monthly traffic to the daily mean:
$$
\begin{align*}
I_{m} = \frac{\bar{C}_{30d}}{\bar{C}_{24h}}
\end{align*}
$$

In [ ]:
from analysis.visualization.characterisation.indices import monthly_index

monthly_index(loader=dl, station_name=station, channel="channels_all")

## Exploratory Plots

Before clustering, we visually explore the temporal patterns of all stations using normalized indices.

**Hourly Indices**

The aggregated hourly indices illustrate a clear contrast between weekdays and weekends on average: weekdays exhibit a pronounced double-peak structure, while weekend profiles are flatter and more evenly distributed. However, the semi-transparent individual curves already indicate substantial variability across stations.

In [ ]:
from analysis.visualization.characterisation.plotting import plot_hourly_indices_all

plot_hourly_indices_all(dl)


We next inspect hourly profiles on a per-station basis. These plots reveal that several stations deviate from the canonical utilitarian or recreational patterns. Examples include stations with a single dominant peak, shifted peak times, or only minor differences between weekday and weekend profiles.

In [ ]:
from analysis.visualization.characterisation.plotting import plot_hourly_indices

for station in dl.get_bicyle_stations():
  plot_hourly_indices(loader=dl, station_name=station, channel="channels_all", interval=None, ylim=(0, 0.2), show_metrics=False)

**Daily Indices**

Daily indices show a clear reduction in traffic on weekends. However, the magnitude of this drop varies substantially between stations, potentially providing an additional discriminator between recreational and utilitarian usage.

In [ ]:
from analysis.visualization.characterisation.plotting import plot_daily_indices_all

plot_daily_indices_all(dl)

In [ ]:
from analysis.visualization.characterisation.plotting import plot_daily_indices

for s in dl.get_bicyle_stations():
    plot_daily_indices(dl, s, ylim=(0.3,1.5), show_metric=False)

**Monthly Indices**

Monthly indices reveal clear differences in seasonality across stations. While all stations exhibit a noticeable reduction in usage during winter, some show an especially pronounced drop, which may indicate predominantly recreational usage. In contrast, other stations maintain relatively stable traffic levels throughout the year, consistent with mainly utilitarian travel patterns.

Notably, a pronounced decrease in traffic is observed in August, which may be explained by the summer holiday period, during which regular commuting activity is reduced.

In [ ]:
from analysis.visualization.characterisation.plotting import plot_monthly_indices_all

plot_monthly_indices_all(dl)

In [ ]:
from analysis.visualization.characterisation.plotting import plot_monthly_indices

for s in dl.get_bicyle_stations():
    plot_monthly_indices(dl, s, ylim=(0.4, 1.6), show_metric=False)

## Feature Engineering

Building on the exploratory plots, we quantify the observed temporal patterns by defining a compact set of **discriminative features** that enable a systematic characterisation of station usage types.

In [ ]:
from analysis.visualization.characterisation.features import build_feature_df

X = build_feature_df(dl)

### Double Peak Index (DPI):

Weekday hourly profiles of some stations exhibit a pronounced double-peak structure, typically associated with morning and evening commuting. The DPI quantifies this behaviour by identifying dominant morning (5–10 h) and evening (14–20 h) peaks and relating their magnitudes to the average midday level (8–14 h). Stations with clear and balanced commuting peaks yield high DPI values, whereas flat or single-peak profiles result in low scores.

Formally, let $p_m$ and $p_e$ denote the magnitudes of the morning and evening peaks at hours $h_m$ and $h_e$, and let $m$ be the average midday traffic level. The DPI is defined as

$$
\begin{align*}
\text{strength} &= \frac{(p_m - m) + (p_e - m)}{2} \\
\text{symmetry} &= 1 - \frac{|p_m - p_e|}{\max(p_m, p_e)} \\
\text{distance} &= \min\!\left(\frac{|h_e - h_m|}{10},\, 1\right) \\
\text{DPI} &= \max\!\left( \text{strength} \cdot \text{symmetry} \cdot \text{distance},\, 0 \right)
\end{align*}
$$

In [ ]:
X_DPI = (
    X
    .select(["station", "DPI"])
    .sort("DPI", descending=True)
)

X_DPI

### Weekend Shape Difference (WSD):

Differences between weekday and weekend hourly traffic patterns provide an additional discriminator between usage types. To capture this effect, we compare the *shape* of the weekday and weekend hourly profiles.

Let $\mathbf{p}^{wd}$ and $\mathbf{p}^{we}$ denote the weekday and weekend hourly profiles, normalised to sum to one. The Weekend Shape Difference is defined as

$$
\text{WSD} =
\left\lVert
\frac{\mathbf{p}^{wd}}{\sum_h p^{wd}_h}
-
\frac{\mathbf{p}^{we}}{\sum_h p^{we}_h}
\right\rVert_2
$$

High values indicate strong differences between weekday and weekend usage patterns, while low values suggest similar diurnal structures.

In [ ]:
X_WSD = (
    X
    .select(["station", "WSD"])
    .sort("WSD", descending=True)
)

X_WSD

### Seasonal Drop Index (SDI):

Seasonality provides a further discriminator between recreational and utilitarian stations. The SDI quantifies the relative drop between high-usage and low-usage months based on the monthly index.

Let $I_m$ denote the monthly index values of a station. Using upper and lower quantiles to ensure robustness, the Seasonal Drop Index is defined as

$$
\begin{align*}
q_{90} &= \text{quantile}_{0.9}(I_m) \\
q_{10} &= \text{quantile}_{0.1}(I_m) \\
\text{SDI} &= \frac{q_{90} - q_{10}}{q_{90}}
\end{align*}
$$

High SDI values indicate strong seasonal variation, while low values correspond to relatively stable, year-round usage.


In [ ]:
X_SDI = (
    X
    .select(["station", "SDI"])
    .sort("SDI", descending=True)
)

X_SDI

## Clustering using K-Means
To group stations based on their extracted features, we apply k-means clustering to the feature space spanned by DPI, WSD, and SDI. Prior to clustering, all features are standardised to ensure equal weighting.

The number of clusters is fixed to $k=3$:

In [ ]:
N_CLUSTERS = 3

reflecting the conceptual distinction between **recreational-oriented**, **utilitarian-oriented**, and **mixed** usage types introduced earlier. Rather than optimising purely geometric separation metrics, this choice prioritises **interpretability** and alignment with the underlying traffic behaviour assumptions.

In [ ]:
from analysis.visualization.characterisation.clustering import kmeans_clustering

features_full = build_feature_df(dl)

clustering = kmeans_clustering(features=features_full, k=N_CLUSTERS)

clustering.drop(["valid"])

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

X_valid = X.filter(pl.col("valid") == True)
X_feat = X_valid.drop(["station", "valid"]).to_numpy()
X_scaled = StandardScaler().fit_transform(X_feat)

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0, n_init=20)
labels = kmeans.fit_predict(X_scaled)

X_valid = X_valid.with_columns(pl.Series("cluster", labels))

X = X.join(
    X_valid.select(["station", "cluster"]),
    on="station",
    how="left"
)

X.drop(["station", "valid"]).group_by("cluster").mean()

### Feature discrimination across clusters

Before we continue let's check how much the features to discriminate between the clusters. To assess this we inspect their distributions using boxplots.

In [ ]:
from analysis.visualization.characterisation.plotting import plot_feature_boxplots

EXCLUDE = {"station", "valid", "cluster", "date"}

FEATURES = [
    c for c in X.columns
    if c not in EXCLUDE and X[c].dtype in (pl.Float32, pl.Float64)
]

plot_feature_boxplots(
    X,
    features=FEATURES,
    cluster_col="cluster",
    clusters=(0, 1, 2),
)

**DPI:** shows clear separation between clusters, with one cluster exhibiting pronounced weekday double-peak structures typical of commuting traffic, while another cluster displays consistently low DPI values, indicating the absence of work-related patterns.

**WSD:** strongly differentiates the clusters, separating stations with substantial weekday–weekend contrasts from those with similar daily profiles throughout the week.

**SDI:** separation is less pronounced, it supports the distinction between clusters with stable year-round usage and those with stronger seasonal dependence.

### Feature correlation

In [ ]:
X.select(FEATURES).corr()

We observe a strong correlation between **DPI** and **WSD**.  
High **DPI** ⇒ pronounced weekday commuting peaks ⇒ strong structural differences between weekday and weekend profiles (absence of commuting peaks on weekends) ⇒ high **WSD**.

Although correlated, the features are not redundant: **DPI** captures the strength of commuting-related peak structure, while **WSD** quantifies the overall weekday–weekend shape difference. 

### PCA
Reducing the standardized feature space to two dimensions using PCA. This allows us to further explore the feature space 
Using the first two components the majority of variance is explained. 

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print("Explained variance:", pca.explained_variance_ratio_)

X_pca_pl = (
    X
    .with_columns([
        pl.Series("PC1", X_pca[:, 0]),
        pl.Series("PC2", X_pca[:, 1]),
    ])
)
X_pca_pl

In [ ]:
from analysis.visualization.characterisation.plotting import plot_pca_clusters

plot_pca_clusters(
    X_pca_pl,
    cluster_col="cluster",
    annotate=True
)

The projection reveals a clear separation of stations primarily along the first principal component (PC1). This axis reflects the contrast between stations with pronounced weekday commuting structure and those with more homogeneous, recreational traffic patterns. 

The second principal component (PC2) captures secondary variations, including differences in seasonal sensitivity and intermediate usage characteristics.

## Probabalistic Clustering using K-Means
So far, station usage types were derived from a single k-means clustering applied to features aggregated over the full observation period of each station.

However, the temporal coverage differs between stations, as shown below.

As a consequence, cluster assignments may depend on the chosen observation window. To assess the temporal robustness of the clustering, we recompute the feature vectors and cluster assignments for a restricted time interval.

In [ ]:
rows = []

for station in dl.get_bicyle_stations():
    bd = dl.get_bicycle(station_name=station)
    min_date, max_date = bd.date_range()
    rows.append({
        "station": station,
        "start_date": min_date,
        "end_date": max_date,
    })

df_dates = pl.DataFrame(rows)

df_dates = df_dates.with_columns(
    (pl.col("end_date") - pl.col("start_date"))
        .dt.total_days()
        .alias("summed_days")
)

df_dates

### Temporal robustness check

We focus on the period 2021–2024, since all stations are continuously observed from 2021 onwards (excluding Ernst-Walz-Brücke West – alt, which was removed in 2018).
The resulting clustering is compared to the clustering obtained from the full data set.

Cluster consistency is quantified using the Adjusted Rand Index (ARI), which measures the agreement between two clusterings while correcting for chance agreement.

In [ ]:
from analysis.visualization.characterisation.clustering import cluster_ari

interval_21_24 = ("2021-01-01", "2024-01-01")
features_21_24 = build_feature_df(dl, interval_21_24)

clustering_full = kmeans_clustering(features=features_full, k=N_CLUSTERS)
clustering_21_24 = kmeans_clustering(features=features_21_24, k=N_CLUSTERS)

score = cluster_ari(clustering_full, clustering_21_24)
print(f"Cluster Consistency Score: {score * 100:.2f}%")

The ARI indicates that cluster assignments are sensitive to the selected time window.
This behaviour is expected, as traffic patterns may evolve over time and aggregated features depend on the available observation period.

Rather than interpreting this variability as a weakness of the feature set, it motivates a probabilistic interpretation of station usage types.

### Probabilistic station classification
Instead of assigning each station to a single fixed cluster, we repeatedly apply k-means clustering over an expanding (cumulative) time window and compute cluster membership probabilities.

For each station $s$ and usage type $u$, we define:

$$
\begin{align*}
P(\;u\;|\;s\;) = \frac{\text{Number of assignments of station s to type u}}{\text{Total number of clustering runs}}
\end{align*}
$$

#### Clustering probabilities

To obtain a probabilistic and temporally robust station classification, k-means clustering is applied repeatedly using either a sliding or a cumulative time window.

- **Sliding window (`sliding`)**: Clustering is performed on a fixed-length window (e.g. 24 months) that is shifted forward month by month.
- **Cumulative window (`cumulative`)**: Clustering is performed on an expanding window with a fixed start date.

In the following, we use a two-year sliding window, as consecutive windows differ only slightly. This allows temporal changes in station usage to be captured more effectively than with the cumulative approach, which increasingly aggregates long-term behaviour.

In [ ]:
from analysis.visualization.characterisation.clustering import monthly_dates, make_interval

DATASET_START = "2016-01-01"
DATASET_END = "2025-01-01"
TIME_SERIES_MODE = "sliding"
WINDOW_MONTHS = 24


dates = monthly_dates(start=DATASET_START, end=DATASET_END)
for d in dates:
  i = make_interval(start=DATASET_START, end=d, mode=TIME_SERIES_MODE, window_months=24)
  if i is None:
    continue
  print(i)

### Cluster Interpretation

K-means assigns arbitrary numeric cluster IDs without semantic meaning.  
To obtain interpretable usage types, clusters are labelled post hoc based on their aggregated feature characteristics.

For each cluster, mean feature values (DPI, WSD, SDI) are computed and standardised.

A scalar utilitarian score is defined as

$$
\text{Utilitarian Score} = \text{DPI} + \text{WSD} - \text{SDI}
$$

Clusters are ordered by this score and labelled as **recreational**, **mixed**, and **utilitarian** from low to high values.  
Labels are derived from the full-period feature representation.

In [ ]:
from analysis.visualization.characterisation.clustering import cluster_timeseries_usage, usage_probabilities

usage = cluster_timeseries_usage(
    loader=dl,
    k=N_CLUSTERS,
    features=FEATURES,
    start=DATASET_START,
    end=DATASET_END,
    mode=TIME_SERIES_MODE,
    window_months=WINDOW_MONTHS
)

usage_probs = usage_probabilities(usage).sort(["station", "probability"], descending=True)

In [ ]:
from analysis.visualization.characterisation.plotting import plot_cluster_probabilities_ci

plot_cluster_probabilities_ci(cluster_probs_ci=usage_probs, station_col="station", prob_col="probability", lo_col="ci_low", hi_col="ci_high")

In [ ]:
top_per_usage_type = (
    usage_probs
    .sort("probability", descending=True)
    .group_by("usage_type")
    .head(20)  
)

top_per_usage_type

### Temporal Stability of Station Usage Types

To quantify the temporal stability of station usage patterns, we compute the Shannon entropy of the cluster membership distribution for each station. For a station $s$ with usage probabilities $P(u \mid s)$, the entropy is defined as

$$
H(s) = -\sum_{u} P(u \mid s)\,\log P(u \mid s)
$$

In [ ]:
from analysis.visualization.characterisation.helpers import entropy, dominant_usage_per_station

entropy_df = entropy(usage_probs=usage_probs)
dominant_usage = dominant_usage_per_station(usage_probs=usage_probs)

entropy_labeled = (
    entropy_df
    .join(dominant_usage, on="station", how="left")
    .sort("entropy")
)

entropy_labeled

Stations with near-deterministic usage probabilities show minimal entropy, indicating stable temporal behaviour.

Higher entropy values occur predominantly for mixed-use stations (see plot below), reflecting structurally variable or context-dependent usage patterns.

In [ ]:
from analysis.visualization.characterisation.plotting import plot_usage_entropy

plot_usage_entropy(entropy_df=entropy_labeled)

### Map View of Clustering
Lets look how the clustering looks on a map.

#### Dynamic Map

In [ ]:
from analysis.visualization.characterisation.map.dynamic_map import bicycle_station_cluster_map
from IPython.display import display, clear_output

clear_output(wait=True)
display(bicycle_station_cluster_map(loader=dl, usage_probs=usage_probs))

#### Static Map

In [ ]:
from analysis.visualization.characterisation.map.static_map import plot_bicycle_usage_map

fig, ax = plot_bicycle_usage_map(
    usage_probs=usage_probs,
    loader=dl,
    city_geojson_path="analysis/visualization/characterisation/map/stadtgrenze_heidelberg.geojson",
    save=False,
    zoom=0.37,
    shift_y=0,
    shift_x= 0.015
)

## Impact of Public Holidays
To quantify the impact of public holidays on station usage patterns, we compare feature vectors computed from holiday periods with a baseline excluding holidays and analyse the resulting changes in the Double Peak Index (ΔDPI) and the Weekend Shape Difference (ΔWSD).


### Station Usage Patterns

In [ ]:

from analysis.visualization.characterisation.event import compute_event_deltas

holiday_intervals = dl.get_all_holiday_intervals(school_vacation=False)

delta_df = (
    compute_event_deltas(loader=dl, intervals=holiday_intervals)
    .select(["station", "DPI_delta", "WSD_delta"])
)

In [ ]:
from analysis.visualization.characterisation.helpers import impact_by_usage, label_deltas_with_usage

delta_labeled = label_deltas_with_usage(delta_df=delta_df, usage_probs=usage_probs)
impact_holiday = impact_by_usage(delta_labeled=delta_labeled)

impact_holiday

In [ ]:
from analysis.visualization.characterisation.plotting import plot_holiday_impact

plot_holiday_impact(delta_labeled=delta_labeled)


Overall effects
- **ΔDPI < 0 for all stations:** Weekday morning and evening commuter peaks are weakened during public holidays.
- **ΔWSD < 0 for all stations:** Weekday traffic patterns become more similar to weekend profiles.

Mixed-use stations exhibit the largest absolute holiday-induced change in feature space

Recreational shows lowest change

### Utilitarian Score of Stations

In [ ]:
from analysis.visualization.characterisation.plotting import plot_event_utilitarian_spline
plot_event_utilitarian_spline(loader=dl, intervals=holiday_intervals, title="Impact of holidays on utilitarian score of stations", k = 2)

## Impact of Weather on Station Usage Patterns

### Event based effects

In [ ]:
rainy = pl.col("precipitation") >= 2.0
hot   = pl.col("temperature_2m") >= 23
cold  = pl.col("temperature_2m") <= 5
windy = pl.col("wind_speed_10m") > 6

wd = dl.get_weather(sample_rate="1d")

rain_intervals = wd.get_intervals(rainy)
hot_intervals = wd.get_intervals(hot)
cold_intervals = wd.get_intervals(cold)
windy_intervals = wd.get_intervals(windy)

In [ ]:
from analysis.visualization.characterisation.weather import compute_weather_deltas

delta_rainy = compute_weather_deltas(loader=dl, event_intervals=rain_intervals, weekday=None, hours=None)
delta_hot = compute_weather_deltas(loader=dl, event_intervals=hot_intervals, weekday=None, hours=None)
delta_cold = compute_weather_deltas(loader=dl, event_intervals=cold_intervals, weekday=None, hours=None)
delta_windy = compute_weather_deltas(loader=dl, event_intervals=windy_intervals, weekday=None, hours=None)

delta_rainy_labeled = label_deltas_with_usage(delta_df=delta_rainy, usage_probs=usage_probs)
delta_hot_labeled = label_deltas_with_usage(delta_df=delta_hot, usage_probs=usage_probs)
delta_cold_labeled = label_deltas_with_usage(delta_df=delta_cold, usage_probs=usage_probs)
delta_windy_labeled = label_deltas_with_usage(delta_df=delta_windy, usage_probs=usage_probs)

In [ ]:
DELTA_REL = "delta_relative"  
DELTA_ABS = "delta_absolute"  

impact_rain = (
    delta_rainy_labeled
    .group_by("usage_type")
    .agg([
        pl.len().alias("n_stations"),
        pl.median(DELTA_REL).alias("delta_median"),
        pl.mean(DELTA_REL).alias("delta_mean"),
        pl.quantile(DELTA_REL, 0.25).alias("q25"),
        pl.quantile(DELTA_REL, 0.75).alias("q75"),
    ])
    .sort("delta_median")
)

impact_hot = (
    delta_hot_labeled
    .group_by("usage_type")
    .agg([
        pl.len().alias("n_stations"),
        pl.median(DELTA_REL).alias("delta_median"),
        pl.mean(DELTA_REL).alias("delta_mean"),
        pl.quantile(DELTA_REL, 0.25).alias("q25"),
        pl.quantile(DELTA_REL, 0.75).alias("q75"),
    ])
    .sort("delta_median")
)

impact_cold = (
    delta_cold_labeled
    .group_by("usage_type")
    .agg([
        pl.len().alias("n_stations"),
        pl.median(DELTA_REL).alias("delta_median"),
        pl.mean(DELTA_REL).alias("delta_mean"),
        pl.quantile(DELTA_REL, 0.25).alias("q25"),
        pl.quantile(DELTA_REL, 0.75).alias("q75"),
    ])
    .sort("delta_median")
)

impact_windy = (
    delta_windy_labeled
    .group_by("usage_type")
    .agg([
        pl.len().alias("n_stations"),
        pl.median(DELTA_REL).alias("delta_median"),
        pl.mean(DELTA_REL).alias("delta_mean"),
        pl.quantile(DELTA_REL, 0.25).alias("q25"),
        pl.quantile(DELTA_REL, 0.75).alias("q75"),
    ])
    .sort("delta_median")
)

In [ ]:
impact_rain

In [ ]:
impact_hot

In [ ]:
impact_cold

In [ ]:
impact_windy

### Continuous responses

In [ ]:
from analysis.visualization.characterisation.plotting import plot_weather_response
from analysis.visualization.characterisation.weather import weather_response_by_usage, weather_response_df

df = weather_response_df(loader=dl)

df = df.join(
    dominant_usage_per_station(usage_probs),
    on="station",
    how="left"
)

In [ ]:
temp_resp = weather_response_by_usage(
    df,
    var="temperature_2m",
    bin_width=0.3,
    baseline_cond=pl.col("temperature_2m") <= 10,
    min_var=10,
    max_var=29,
)

plot_weather_response(
    temp_resp,
    xlabel="Temperature (°C)",
    title="Temperature response by usage type",
)


In [ ]:
wind_resp = weather_response_by_usage(
    df,
    var="wind_speed_10m",
    bin_width=0.3,
    baseline_cond=pl.col("wind_speed_10m") <= 2,
    min_var=0.1,
    max_var=10,
)

plot_weather_response(
    wind_resp,
    xlabel="Wind speed (m/s)",
    title="Wind response by usage type",
)


In [ ]:
rain_resp = weather_response_by_usage(
    df,
    var="precipitation",
    bin_width=0.1,
    baseline_cond=pl.col("precipitation") == 0,
    min_var=0,
    max_var=2.5,
)

plot_weather_response(
    rain_resp,
    xlabel="Precipitation (mm)",
    title="Rain response by usage type",
)


## Result: Same effect sequence as with public holidays

### mixed > utilitarian > recreational